# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [76]:
# importando o selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# abrindo navegador
navegador = webdriver.Chrome()

# dolar
navegador.get('https://www.google.com/')

navegador.find_element(by=By.XPATH, value='/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')

navegador.find_element(by=By.XPATH, value='/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = float(navegador.find_element(by=By.XPATH, value='//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value'))

# euro
navegador.get('https://www.google.com/')

navegador.find_element(by=By.XPATH, value='/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')

navegador.find_element(by=By.XPATH, value='/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = float(navegador.find_element(by=By.XPATH, value='//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value'))

# ouro
navegador.get('https://dolarhoje.com/ouro-hoje/')

cotacao_ouro = float(navegador.find_element(by=By.XPATH, value='//*[@id="nacional"]').get_attribute('value').replace(',', '.'))

navegador.quit()

- Atualizando os preços e o cálculo do Preço Final

In [77]:
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')

tabela['Cotação'] = tabela['Cotação'].astype(float)

tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.040300,5040.249597,1.40,7056.349436
1,Carro Renault,4500.00,Euro,5.591961,25163.823758,2.00,50327.647515
2,Notebook Dell,899.99,Dólar,5.040300,4536.219597,1.70,7711.573315
3,IPhone,799.00,Dólar,5.040300,4027.199700,1.70,6846.239490
4,Carro Fiat,3000.00,Euro,5.591961,16775.882505,1.90,31874.176760
5,Celular Xiaomi,480.48,Dólar,5.040300,2421.763344,2.00,4843.526688
6,Joia 20g,20.00,Ouro,317.830000,6356.600000,1.15,7310.090000


### Agora vamos exportar a nova base de preços atualizada

In [78]:
tabela.to_excel('Produtos Novo.xlsx', index=False)